<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Yolo2Rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import shutil
import json

# Rutas de las carpetas de imágenes y etiquetas originales
images_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/images"
labels_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/labels"

# Ruta principal del dataset final
main_dir = "/content/dataset_MaskRCNN"

# Subcarpetas del dataset final
train_dir = os.path.join(main_dir, "train")
test_dir = os.path.join(main_dir, "test")
val_dir = os.path.join(main_dir, "val")

# Rutas de las carpetas de train, test y val para imágenes y etiquetas
train_images_dir = os.path.join(train_dir, "images")
train_labels_dir = os.path.join(train_dir, "labels")
test_images_dir = os.path.join(test_dir, "images")
test_labels_dir = os.path.join(test_dir, "labels")
val_images_dir = os.path.join(val_dir, "images")
val_labels_dir = os.path.join(val_dir, "labels")

# Función para crear carpetas si no existen
def create_directories_if_not_exist(*directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Crear carpetas de train, test y val si no existen
create_directories_if_not_exist(train_dir, test_dir, val_dir)
create_directories_if_not_exist(train_images_dir, train_labels_dir, test_images_dir, test_labels_dir, val_images_dir, val_labels_dir)

# Proporciones para train, test y val
train_percent = 0.6
test_percent = 0.2
val_percent = 0.2

# Obtener lista de nombres de archivos de imágenes
image_files = os.listdir(images_dir)
# Filtrar solo los archivos con extensión .jpg
image_files = [file for file in image_files if file.endswith(".jpg")]

# Barajar la lista de nombres de archivos
random.shuffle(image_files)

# Calcular el número de archivos para cada conjunto
total_images = len(image_files)
num_train = int(total_images * train_percent)
num_test = int(total_images * test_percent)
num_val = total_images - num_train - num_test

# Dividir la lista de nombres de archivos en train, test y val
train_images = image_files[:num_train]
test_images = image_files[num_train:num_train+num_test]
val_images = image_files[num_train+num_test:]

# Crear carpetas de train, test y val si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Copiar imágenes y etiquetas a las carpetas correspondientes
def copy_files(files, source_dir, dest_images_dir, dest_labels_dir):
    for file in files:
        # Copiar imágenes
        shutil.copy(os.path.join(source_dir, file), dest_images_dir)
        # Copiar etiquetas
        label_file = os.path.splitext(file)[0] + ".txt"
        shutil.copy(os.path.join(labels_dir, label_file), dest_labels_dir)

copy_files(train_images, images_dir, train_images_dir, train_labels_dir)
copy_files(test_images, images_dir, test_images_dir, test_labels_dir)
copy_files(val_images, images_dir, val_images_dir, val_labels_dir)

print("División de imágenes y etiquetas completada.")


División de imágenes y etiquetas completada.


# Definimos las funciones de lectura y cambio de formato de las etiquetas

In [3]:
def read_yolo_labels(yolo_label_dir):
    """Función para leer archivos .txt de etiquetas de YOLO."""
    yolo_labels = {}
    for file_name in os.listdir(yolo_label_dir):
        if file_name.endswith(".txt"):
            with open(os.path.join(yolo_label_dir, file_name), "r") as f:
                content = f.read().strip().split("\n")
                labels = []
                for line in content:
                    parts = line.split(" ")
                    class_id = int(parts[0])
                    points = []
                    for i in range(1, len(parts), 2):
                        x, y = map(float, parts[i:i+2])
                        point = [x, y]
                        points.append(point)
                    label = {"class_id": class_id, "points": points}
                    labels.append(label)
                yolo_labels[file_name] = labels
    return yolo_labels

def convert_to_mask_rcnn_labels(yolo_labels):
    """Convierte un archivo .txt de etiquetas de segmentación de Yolo en un .json de etiquetas de segmentación de Mask RCNN."""
    mask_rcnn_labels = {}
    for file_name, yolo_objs in yolo_labels.items():
        mask_rcnn_labels[file_name.replace(".txt", ".jpg")] = {
            "fileref": "",
            "size": 123456,  # Tamaño de la imagen (ajustar según sea necesario)
            "filename": file_name.replace(".txt", ".jpg"),
            "base64_img_data": "",
            "file_attributes": {},
            "regions": {}
        }
        for i, obj in enumerate(yolo_objs):
            mask_rcnn_labels[file_name.replace(".txt", ".jpg")]["regions"][str(i)] = {
                "shape_attributes": {
                    "name": "polygon",
                    "all_points_x": [int(point[0] * 123) for point in obj["points"]],  # Ajustar a las dimensiones de la imagen
                    "all_points_y": [int(point[1] * 456) for point in obj["points"]]  # Ajustar a las dimensiones de la imagen
                },
                "region_attributes": {
                    "class": obj["class_id"]  # Clase del objeto
                }
            }
    return mask_rcnn_labels

def merge_mask_rcnn_labels(input_dir, output_dir):
    """Se indica el directorio que contiene los .json que se quieren juntar en un solo archivo y se indica la ruta donde se quiere guardar dicho archivo."""
    # Verificar si el directorio de salida existe, si no, crearlo
    os.makedirs(output_dir, exist_ok=True)

    # Diccionario para almacenar todos los diccionarios de imágenes
    merged_data = {}

    # Iterar sobre cada archivo en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(input_dir, filename)
            # Abrir y cargar cada archivo JSON
            with open(filepath, "r") as f:
                data = json.load(f)
            # Extraer el nombre del archivo (sin la extensión)
            filename_key = os.path.splitext(filename)[0]
            # Agregar los datos de la imagen al diccionario fusionado
            merged_data[filename_key] = data

    # Escribir el diccionario fusionado en un archivo JSON en el directorio de salida
    output_file = os.path.join(output_dir, "merged_labels.json")
    with open(output_file, "w") as f:
        json.dump(merged_data, f, indent=4)

# Leemos las etiquetas en formato YoloV9


In [4]:
# Leer archivos de etiquetas de YOLO para train, test y val
train_yolo_labels = read_yolo_labels(train_labels_dir)
test_yolo_labels = read_yolo_labels(test_labels_dir)
val_yolo_labels = read_yolo_labels(val_labels_dir)

print("Etiquetas de YOLO leídas para train, test y val.")


Etiquetas de YOLO leídas para train, test y val.


# Convertimos las etiquetas en formato Mask RCNN

In [5]:
# Carpeta de train
train_mask_rcnn_labels = convert_to_mask_rcnn_labels(train_yolo_labels)
train_RCNN_labels_dir = "/content/dataset_MaskRCNN/train/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(train_RCNN_labels_dir):
    os.makedirs(train_RCNN_labels_dir)

for file_name, label_data in train_mask_rcnn_labels.items():
    with open(os.path.join(train_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)

# Carpeta de test
test_mask_rcnn_labels = convert_to_mask_rcnn_labels(test_yolo_labels)
test_RCNN_labels_dir = "/content/dataset_MaskRCNN/test/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(test_RCNN_labels_dir):
    os.makedirs(test_RCNN_labels_dir)

for file_name, label_data in test_mask_rcnn_labels.items():
    with open(os.path.join(test_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)

# Carpeta de val
val_mask_rcnn_labels = convert_to_mask_rcnn_labels(val_yolo_labels)
val_RCNN_labels_dir = "/content/dataset_MaskRCNN/val/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(val_RCNN_labels_dir):
    os.makedirs(val_RCNN_labels_dir)

for file_name, label_data in val_mask_rcnn_labels.items():
    with open(os.path.join(val_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)


# Juntamos todas las etiquetas en un solo .json para cada carpeta de Train, Test y Val.

In [6]:
# Rutas de los directorios de train, test y val
train_RCNN_labels_dir = "/content/dataset_MaskRCNN/train/RCNN_labels"
test_RCNN_labels_dir = "/content/dataset_MaskRCNN/test/RCNN_labels"
val_RCNN_labels_dir = "/content/dataset_MaskRCNN/val/RCNN_labels"

# Rutas de los directorios de salida fusionados
merged_train_RCNN_labels_dir = "/content/dataset_MaskRCNN/train/RCNN_labels_merged/"
merged_test_RCNN_labels_dir = "/content/dataset_MaskRCNN/test/RCNN_labels_merged/"
merged_val_RCNN_labels_dir = "/content/dataset_MaskRCNN/val/RCNN_labels_merged/"

# Crear los directorios de salida si no existen
create_directories_if_not_exist(merged_train_RCNN_labels_dir, merged_test_RCNN_labels_dir, merged_val_RCNN_labels_dir)

# Llamar a la función para fusionar los archivos JSON
merge_mask_rcnn_labels(train_RCNN_labels_dir, merged_train_RCNN_labels_dir)
merge_mask_rcnn_labels(test_RCNN_labels_dir, merged_test_RCNN_labels_dir)
merge_mask_rcnn_labels(val_RCNN_labels_dir, merged_val_RCNN_labels_dir)